In [1]:
import os, numpy as np
import pandas as pd

os.environ["KERAS_BACKEND"] = "cntk"

import keras 
from sklearn import datasets

from sklearn.model_selection import train_test_split

from keras.utils import np_utils


keras.backend.set_floatx('float32')

from keras.utils import np_utils


def set_reproducible():
    import tensorflow as tf
    import random as rn
    import os
    os.environ['PYTHONHASHSEED'] = '1960'
    rn.seed(1960)
    np.random.seed(1960)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K
    tf.set_random_seed(1960)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    # K.set_session(sess)

set_reproducible()


Using CNTK backend
/home/antoine/.local/lib/python3.6/site-packages/keras/backend/cntk_backend.py:22: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


# Build a Keras Model

In [2]:

def create_model():
    from keras.models import Sequential
    from keras.layers.core import Dense, Activation, Dropout
    from keras.utils import np_utils

    model = Sequential()
    model.add(Dense(5, input_shape=(4,) , activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(3))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model



In [3]:
iris  = datasets.load_iris()
iris.data = iris.data.astype(np.float32)
train_X, test_X, train_y, test_y = train_test_split(iris.data, iris.target, train_size=0.8, test_size=0.2, random_state=1960)


In [4]:

from keras.wrappers.scikit_learn import KerasClassifier

clf = KerasClassifier(build_fn=create_model, epochs=12, verbose=0)

print(train_X.shape , train_y.shape)
clf.fit(train_X, train_y, verbose=0)

(120, 4) (120,)


In [5]:
print(clf.model.__dict__)

{'name': 'sequential_1', 'trainable': True, '_is_compiled': True, '_expects_training_arg': False, '_initial_weights': None, 'supports_masking': False, 'optimizer': <keras.optimizers.Adam object at 0x7f7d08583358>, '_updates': [], '_losses': [], '_per_input_losses': {}, '_per_input_updates': {}, '_layers': [<keras.engine.input_layer.InputLayer object at 0x7f7d085d1668>, <keras.layers.core.Dense object at 0x7f7d085d1c18>, <keras.layers.core.Dropout object at 0x7f7d085d1a20>, <keras.layers.core.Dense object at 0x7f7d085d16a0>, <keras.layers.core.Activation object at 0x7f7d08583208>], '_outbound_nodes': [], '_inbound_nodes': [<keras.engine.base_layer.Node object at 0x7f7d085d16d8>], '_is_graph_network': True, '_uses_inputs_arg': True, 'outputs': [Composite(Softmax): Input('dense_1_input', [#], [4]) -> Output('Softmax43_Output_0', [#], [3])], 'inputs': [Input('dense_1_input', [#], [4])], '_built': True, '_compute_previous_mask': True, '_input_layers': [<keras.engine.input_layer.InputLayer o

In [6]:
print(test_X.shape)
preds = clf.predict(test_X[0,:].reshape(1,4))
print(preds)


(30, 4)
[2]


# Generate SQL Code from the Model

In [7]:
import json, requests, base64, dill as pickle, sys



sys.setrecursionlimit(200000)
pickle.settings['recurse'] = False

# no luck for the web service... pickling feature of tensorflow and/or keras objects seems not to be a priority.
# there is a lot of github issues in the two projects when I search for pickle keyword!!!.

def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



def test_sql_gen(keras_regressor , metadata):
    import sklearn2sql.PyCodeGenerator as codegen
    cg1 = codegen.cAbstractCodeGenerator();
    cg1.mOptions.mActivateMaterialization = False
    lSQL = cg1.generateCodeWithMetadata(clf, metadata, dsn = None, dialect = "postgresql");
    return lSQL[1]


In [8]:
# commented .. see above
# pickle_data = pickle.dumps(clf)
# lSQL = test_ws_sql_gen(pickle_data)
# print(lSQL[0:2000])

In [9]:
lMetaData = {}
lMetaData['features'] = list(iris.feature_names);
#column name adaptation
lMetaData['features'] = [col.replace(" (cm)" , "_cm") for col in lMetaData['features'] ]

lMetaData["targets"] = ['TGT']
lMetaData['primary_key'] = 'KEY'
lMetaData['table'] = 'iris'

    
lSQL = test_sql_gen(clf , lMetaData)


cGenerationWrapperFactory::createWrapper() <class 'keras.wrappers.scikit_learn.KerasClassifier'>
cClassifierMixin_CodeGenWrapper::setObject <class 'keras.wrappers.scikit_learn.KerasClassifier'>
BACKEND_DIALECT postgresql
CREATING_DATABASE_BACKEND_DSN_DIALECT 1.2.7 None postgresql
KERAS_GENERATE_EXPRESSION_START <class 'sqlalchemy.sql.selectable.Alias'> <class 'sklearn2sql.GenericModels.Keras.cSerializedKeras'>
{'mKerasData': <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f7d085d1cf8>, 'mFeatureNames': ['sepal length_cm', 'sepal width_cm', 'petal length_cm', 'petal width_cm'], 'mClasses': array([0, 1, 2]), 'mMode': 'Classification'}
{'build_fn': <function create_model at 0x7f7d085d9f28>, 'sk_params': {'epochs': 12, 'verbose': 0}, 'classes_': array([0, 1, 2]), 'n_classes_': 3, 'model': <keras.engine.sequential.Sequential object at 0x7f7d085d1b70>}
ABSTRACT_GENERATE_CTE keras_input keras_input 5 False False
ABSTRACT_GENERATE_SIMPLE_CTE keras_input 5
GENERATING_LAYER 0 dense_1 <

In [10]:
print(lSQL)

WITH keras_input AS 
(SELECT "ADS"."KEY" AS "KEY", "ADS"."sepal length_cm" AS "sepal length_cm", "ADS"."sepal width_cm" AS "sepal width_cm", "ADS"."petal length_cm" AS "petal length_cm", "ADS"."petal width_cm" AS "petal width_cm" 
FROM iris AS "ADS"), 
keras_input_1 AS 
(SELECT keras_input."KEY" AS "KEY", keras_input."sepal length_cm" AS "sepal length_cm", keras_input."sepal width_cm" AS "sepal width_cm", keras_input."petal length_cm" AS "petal length_cm", keras_input."petal width_cm" AS "petal width_cm" 
FROM keras_input), 
layer_dense_1 AS 
(SELECT keras_input_1."KEY" AS "KEY", -0.042614053934812546 + 0.22980284690856934 * keras_input_1."sepal length_cm" + 0.4737735390663147 * keras_input_1."sepal width_cm" + -0.7232964038848877 * keras_input_1."petal length_cm" + -0.5342894196510315 * keras_input_1."petal width_cm" AS output_1, 0.0 + 0.06646186113357544 * keras_input_1."sepal length_cm" + -0.7361471652984619 * keras_input_1."sepal width_cm" + -0.8049471378326416 * keras_input_1."pet

# Execute the SQL Code

In [11]:
# save the dataset in a database table


import sqlalchemy as sa

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(iris.data);
lTable.columns = lMetaData['features']
lTable['TGT'] = iris.target
lTable['KEY'] = range(iris.data.shape[0])
lTable.to_sql(lMetaData['table'] , conn,   if_exists='replace', index=False)

In [12]:
sql_output = pd.read_sql(lSQL , conn);
conn.close()

In [13]:
sql_output.sample(12, random_state=1960)

,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,None,None,None,0.324453,0.214350,0.461197,-1.125615,-1.540146,-0.773930,2,0.461197
74,74,None,None,None,0.335966,0.340661,0.323374,-1.090747,-1.076869,-1.128946,1,0.340661
9,9,None,None,None,0.254071,0.409067,0.336862,-1.370140,-0.893877,-1.088082,1,0.409067
88,88,None,None,None,0.335966,0.340661,0.323374,-1.090747,-1.076869,-1.128946,1,0.340661
25,25,None,None,None,0.262164,0.401868,0.335967,-1.338784,-0.911631,-1.090741,1,0.401868
5,5,None,None,None,0.244108,0.418084,0.337808,-1.410144,-0.872073,-1.085277,1,0.418084
48,48,None,None,None,0.237207,0.424435,0.338358,-1.438822,-0.856997,-1.083650,1,0.424435
117,117,None,None,None,0.320468,0.200192,0.479339,-1.137972,-1.608477,-0.735346,2,0.479339
83,83,None,None,None,0.326860,0.224292,0.448848,-1.118222,-1.494808,-0.801071,2,0.448848
105,105,None,None,None,0.304021,0.157835,0.538144,-1.190658,-1.846207,-0.619629,2,0.538144


# Keras Prediction

In [14]:
keras_output = pd.DataFrame()
keras_output_key = pd.DataFrame(list(range(iris.data.shape[0])), columns=['KEY']);
keras_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
keras_output_proba = pd.DataFrame(clf.predict_proba(iris.data), columns=['Proba_0', 'Proba_1', 'Proba_2'])
keras_output = pd.concat([keras_output_key, keras_output_score, keras_output_proba] , axis=1)
for class_label in [0, 1, 2]:
    keras_output['LogProba_' + str(class_label)] = np.log(keras_output_proba['Proba_' + str(class_label)])
keras_output['Decision'] = clf.predict(iris.data)
keras_output.sample(12, random_state=1960)


,KEY,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.324453,0.214350,0.461197,-1.125615,-1.540146,-0.773930,2
74,74,NaN,NaN,NaN,0.335966,0.340661,0.323374,-1.090747,-1.076869,-1.128946,1
9,9,NaN,NaN,NaN,0.254071,0.409067,0.336862,-1.370140,-0.893877,-1.088083,1
88,88,NaN,NaN,NaN,0.335966,0.340661,0.323374,-1.090747,-1.076869,-1.128946,1
25,25,NaN,NaN,NaN,0.262164,0.401868,0.335967,-1.338784,-0.911631,-1.090741,1
5,5,NaN,NaN,NaN,0.244108,0.418084,0.337808,-1.410144,-0.872074,-1.085278,1
48,48,NaN,NaN,NaN,0.237207,0.424435,0.338358,-1.438822,-0.856997,-1.083650,1
117,117,NaN,NaN,NaN,0.320468,0.200192,0.479339,-1.137972,-1.608477,-0.735346,2
83,83,NaN,NaN,NaN,0.326860,0.224292,0.448848,-1.118222,-1.494808,-0.801071,2
105,105,NaN,NaN,NaN,0.304021,0.157835,0.538144,-1.190658,-1.846207,-0.619629,2


# Comparing the SQL and Keras Predictions

In [15]:
sql_keras_join = keras_output.join(sql_output , how='left', on='KEY', lsuffix='_keras', rsuffix='_sql')


In [16]:
sql_keras_join.head(12)

,KEY_keras,Score_0_keras,Score_1_keras,Score_2_keras,Proba_0_keras,Proba_1_keras,Proba_2_keras,LogProba_0_keras,LogProba_1_keras,LogProba_2_keras,...,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql,DecisionProba
0,0,NaN,NaN,NaN,0.240719,0.421192,0.338089,-1.424126,-0.864667,-1.084445,...,None,None,0.240719,0.421192,0.338089,-1.424126,-0.864667,-1.084445,1,0.421192
1,1,NaN,NaN,NaN,0.255594,0.407704,0.336702,-1.364164,-0.897214,-1.088557,...,None,None,0.255594,0.407704,0.336702,-1.364164,-0.897214,-1.088557,1,0.407704
2,2,NaN,NaN,NaN,0.249156,0.413494,0.337351,-1.389678,-0.883113,-1.086632,...,None,None,0.249156,0.413494,0.337351,-1.389678,-0.883113,-1.086633,1,0.413494
3,3,NaN,NaN,NaN,0.260655,0.403202,0.336143,-1.344557,-0.908317,-1.090220,...,None,None,0.260655,0.403202,0.336142,-1.344557,-0.908317,-1.090220,1,0.403202
4,4,NaN,NaN,NaN,0.239462,0.422350,0.338188,-1.429360,-0.861922,-1.084153,...,None,None,0.239462,0.422350,0.338188,-1.429360,-0.861922,-1.084153,1,0.422350
5,5,NaN,NaN,NaN,0.244108,0.418084,0.337808,-1.410144,-0.872074,-1.085278,...,None,None,0.244108,0.418084,0.337808,-1.410144,-0.872073,-1.085277,1,0.418084
6,6,NaN,NaN,NaN,0.252013,0.410916,0.337072,-1.378276,-0.889367,-1.087460,...,None,None,0.252013,0.410916,0.337072,-1.378276,-0.889367,-1.087460,1,0.410916
7,7,NaN,NaN,NaN,0.248153,0.414402,0.337445,-1.393710,-0.880919,-1.086353,...,None,None,0.248153,0.414402,0.337445,-1.393710,-0.880919,-1.086353,1,0.414402
8,8,NaN,NaN,NaN,0.264379,0.399917,0.335704,-1.330372,-0.916498,-1.091526,...,None,None,0.264379,0.399917,0.335704,-1.330372,-0.916498,-1.091526,1,0.399917
9,9,NaN,NaN,NaN,0.254071,0.409067,0.336862,-1.370140,-0.893877,-1.088083,...,None,None,0.254071,0.409067,0.336862,-1.370140,-0.893877,-1.088082,1,0.409067


In [17]:
condition = (sql_keras_join.Decision_sql != sql_keras_join.Decision_keras)
sql_keras_join[condition]

,KEY_keras,Score_0_keras,Score_1_keras,Score_2_keras,Proba_0_keras,Proba_1_keras,Proba_2_keras,LogProba_0_keras,LogProba_1_keras,LogProba_2_keras,...,Score_1_sql,Score_2_sql,Proba_0_sql,Proba_1_sql,Proba_2_sql,LogProba_0_sql,LogProba_1_sql,LogProba_2_sql,Decision_sql,DecisionProba
